Переписываю предыдущую логику в bayes.Inference.py

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
%matplotlib notebook
import numpy as np

import moments
# In demographic_models.py, we've defined a custom model for this problem
import demographic_models

import gadma

import GPy
from GPyOpt.methods import BayesianOptimization

from functools import partial

import pandas as pd

import matplotlib.pyplot as plt

import sys  
sys.path.insert(0, './bayes')
sys.path.insert(0, '../examples')
import Inference as Inf

In [3]:
# Load the data
data = moments.Spectrum.from_file('YRI_CEU.fs')

In [4]:
upper_bound = np.array([100, 100, 100, 10, 3, 3])
lower_bound = np.array([1e-2, 1e-2, 1e-2, 0, 0, 0])

In [5]:
def calc(is_log, need_check, parameters_pos, initial_points_num=5, iter_cnt=100, Y_eps=1e-3, max_time=10, max_iter=100, max_steps=100):
    tmp = np.array2string(parameters_pos)
    
    print('Calculation for %s started' % tmp)
    
    p0 = np.array([[generate_random_value(low_bound, upp_bound) for (low_bound, upp_bound) in zip(lower_bound, upper_bound)]
                   for _ in range(initial_points_num)])
    if is_log:
        p0 = np.log(p0)
    bo = BayesianOptimization(f=partial(obj_func, is_log, parameters_pos),
                              domain=dom_log[parameters_pos] if is_log else domain[parameters_pos],
                              model_type='GP',
                              kernel=GPy.kern.Matern52(input_dim=parameters_pos.shape[0]),
                              acquisition_type ='EI',
                              X=p0,
                              ARD=True
                             )
    
    filename = '%d_%s_%s.csv' % (max_time, tmp, 'log' if is_log else '')
    eval_filename = EVALS + filename
    model_filename = MODELS + filename
    
    num_steps = 0
    while (not need_check) or check_to_stop(bo, iter_cnt, Y_eps):
        bo.run_optimization(
#                             max_time=max_time,
                            max_iter=max_iter, # TODO: В bo.py 0 по дефолту
#                             evaluations_file=eval_filename, # models_file=model_filename, report_file='tmp.csv',
                            verbosity=True)
        num_steps += bo.num_acquisitions
        print('\t Intermediate value %f at %d iter' % (bo.fx_opt, num_steps))
        if num_steps >= max_steps:
            print('Iterations ended')
            break
#     bo.run_optimization(max_time=max_time, max_iter=max_iter, eps = tolerance, verbosity=True)
    print('\t Calculation for %s done' % tmp)
    return bo

In [11]:
def draw(is_log, bo):
#     bo.plot_acquisition()
    bo.plot_convergence()
    
    x_opt = np.exp(bo.x_opt) if is_log else bo.x_opt
    x_str = np.array2string(x_opt, precision=3, separator=', ', max_line_width=np.inf)
    print(f"The minumum value obtained by the function was {-bo.fx_opt:.4f} \nx = {x_str}")

In [7]:
func = demographic_models.prior_onegrow_mig

In [8]:
bo = Inf.optimize_bayes(data, func, lower_bound, upper_bound, 
                        max_iter=2,
                        kern_func_name='Matern52',
                        output_log_file='BOtmp.log')

In [9]:
# import gadma

# gadma.Inference.optimize_ga(6, data, func,
#                             size_of_generation_in_ga=10,
#                             lower_bound=lower_bound,
#                             upper_bound=upper_bound,
#                             output_log_file='GAtmp.log'
#                            )

In [12]:
draw (True, bo)

The minumum value obtained by the function was -6264.0510 
x = [0.687, 0.091, 3.904, 0.034, 0.006, 0.018]
